# LSM Calculation for OB and NB
- using new users as users's account date within one year

In [2]:
# add answers data
# Import Modules
import pandas as pd
import numpy as np
import sqlite3
import pickle
import os
import matplotlib.pyplot as plt

# Import Dataset
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date, owner_user_id, body
FROM answers
WHERE creation_date > '2021-09-01'
AND creation_date < '2023-09-01';
'''
ans_df = pd.read_sql_query(query, conn)
conn.close()

# Import Users Data
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date
FROM users;
'''
user_df = pd.read_sql_query(query, conn)
conn.close()
user_df = user_df.rename(columns = {'creation_date':'account_creation_date',
                                    'id': 'owner_user_id'})
ans_df = ans_df.dropna(subset = 'owner_user_id')
ans_df['owner_user_id'] = ans_df['owner_user_id'].astype(int).astype(str)
# Merge answers to the correspond users
user_df['owner_user_id'] = user_df['owner_user_id'].astype(str)
ans_df = pd.merge(ans_df, user_df, on = 'owner_user_id', how = 'left')
ans_df

,id,creation_date,owner_user_id,body,account_creation_date
0,69006420,2021-09-01 00:00:18.070,7332046,"<p>The short approach, remove python3.9 from y...",2016-12-22 19:33:51.100
1,69006421,2021-09-01 00:00:28.823,14471093,"<p>open the terminal, run as administrator.</p...",2020-10-18 06:29:51.557
2,69006422,2021-09-01 00:00:35.537,523612,"<p>Conceptually, <code>Base</code> has no reas...",2010-11-29 07:44:17.673
3,69006429,2021-09-01 00:01:09.140,11269158,<p><code>calldata</code> is a special data loc...,2019-03-27 23:15:22.047
4,69006430,2021-09-01 00:01:17.920,2096113,<p>You could do something like this if the str...,2013-02-21 15:43:28.953
...,...,...,...,...,...
3102831,77019850,2023-08-31 23:56:07.187,313768,<p>Make an inner solution routine that accepts...,2010-04-11 04:47:56.083
3102832,77019851,2023-08-31 23:56:19.123,13296852,<p>When you pass <code>stack[100]</code> as an...,2020-04-12 21:26:44.880
3102833,77019853,2023-08-31 23:56:47.647,5103949,"<p>I had the same error, I fixed it by followi...",2015-07-10 17:53:49.643
3102834,77019855,2023-08-31 23:57:36.797,16844882,"<p>In your Razor component, you can inject the...",2021-09-06 16:29:07.503


In [3]:
# Compute difference between post creation_date - user creation_date.
ans_df['month_diff'] = (pd.to_datetime(ans_df['creation_date']) - pd.to_datetime(ans_df['account_creation_date'])) / pd.Timedelta(days=30)
ans_df = ans_df[ans_df['month_diff'] >= 0]
ans_df['year_month_day'] = pd.to_datetime(ans_df['creation_date']).dt.to_period('D').astype(str)
ans_df['account_year_month'] = pd.to_datetime(ans_df['account_creation_date']).dt.to_period('M')
df_new_a = ans_df[ans_df['account_year_month'] >= '2020-09-01']
df_old_a = ans_df[ans_df['account_year_month'] < '2020-09-01']

/tmp/ipykernel_1898953/1871056578.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans_df['year_month_day'] = pd.to_datetime(ans_df['creation_date']).dt.to_period('D').astype(str)
/tmp/ipykernel_1898953/1871056578.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans_df['account_year_month'] = pd.to_datetime(ans_df['account_creation_date']).dt.to_period('M')


In [4]:
print("length of old_df is: ", len(df_new_a))
print("length of new_df is: ", len(df_old_a))

length of old_df is:  822701
length of new_df is:  2280115


### Preprocess

In [6]:
# erase NAs
df_old = df_old_a.dropna(subset=['owner_user_id'])
# Convert types
df_old['owner_user_id'] = df_old['owner_user_id'].astype(int).astype(str)
df_old['creation_date'] = pd.to_datetime(df_old['creation_date'])

# add year_month variable
df_old['year_month'] = df_old['creation_date'].dt.to_period('M')

# Get the total number of unique months
total_months = df_old['year_month'].nunique()
# Count the number of unique months for each user
user_month_counts = df_old.groupby('owner_user_id')['year_month'].nunique()
# Filter users who have written posts in every single month
users_in_every_month = user_month_counts[user_month_counts == total_months].index.tolist()

# Filtered data
df_filtered = df_old[df_old['owner_user_id'].isin(users_in_every_month)]
df_filtered

,id,creation_date,owner_user_id,body,account_creation_date,month_diff,year_month_day,account_year_month,year_month
6,69006435,2021-09-01 00:02:37.350,869736,<p>The error is correct. <code>getInputStream...,2011-07-29 16:32:37.833,122.877083,2021-09-01,2011-07,2021-09
8,69006438,2021-09-01 00:03:36.527,8690857,<p>You should not use <code>.map</code> for si...,2017-09-28 15:59:11.263,47.777880,2021-09-01,2017-09,2021-09
12,69006446,2021-09-01 00:04:33.053,16406,<p>The reason you need line B is called <a hre...,2008-09-17 16:54:47.547,157.709948,2021-09-01,2008-09,2021-09
16,69006451,2021-09-01 00:05:09.193,478884,<p>For example you could do it this way:</p>\n...,2010-10-18 01:02:40.513,132.365335,2021-09-01,2010-10,2021-09
29,69006475,2021-09-01 00:09:56.233,12957340,<p>Here is another potential solution using ti...,2020-02-25 02:55:36.460,18.462832,2021-09-01,2020-02,2021-09
...,...,...,...,...,...,...,...,...,...
3102806,77019800,2023-08-31 23:36:49.467,2156621,<p><code>StreamZip</code> + <code>expand</code...,2013-03-11 11:39:07.450,127.516613,2023-08-31,2013-03,2023-08
3102818,77019822,2023-08-31 23:44:48.760,209103,"<p>The <a href=""https://firebase.google.com/do...",2009-11-11 22:30:27.020,168.035055,2023-08-31,2009-11,2023-08
3102820,77019826,2023-08-31 23:46:31.700,939860,<p>You don't need a &quot;common key&quot; for...,2011-09-12 04:04:08.520,145.727370,2023-08-31,2011-09,2023-08
3102823,77019833,2023-08-31 23:49:08.340,589924,<p>The <code>sprintf</code> operator behaves l...,2011-01-26 00:02:36.877,153.366355,2023-08-31,2011-01,2023-08


In [7]:
len(users_in_every_month)

769

In [8]:
# erase NAs
df_new = df_new_a.dropna(subset=['owner_user_id'])
# Convert types
df_new['owner_user_id'] = df_new['owner_user_id'].astype(int).astype(str)
df_new['creation_date'] = pd.to_datetime(df_new['creation_date'])

# add year_month variable
df_new['year_month'] = df_new['creation_date'].dt.to_period('M')

# Get the total number of unique months
total_months = df_new['year_month'].nunique()
# Count the number of unique months for each user
user_month_counts = df_new.groupby('owner_user_id')['year_month'].nunique()
# Filter users who have written posts in every single month
users_in_every_month = user_month_counts[user_month_counts == total_months].index.tolist()

# Filtered data
df_filtered2 = df_new[df_new['owner_user_id'].isin(users_in_every_month)]
df_filtered2

,id,creation_date,owner_user_id,body,account_creation_date,month_diff,year_month_day,account_year_month,year_month
132,69006762,2021-09-01 01:11:17.457,15261315,"<p>As pointed out in the comments, the means o...",2021-02-22 17:00:43.327,6.344689,2021-09-01,2021-02,2021-09
301,69007160,2021-09-01 02:29:16.003,14868997,<p>The correct way to compare intervals to che...,2020-12-22 02:20:12.663,8.433543,2021-09-01,2020-12,2021-09
1074,69009000,2021-09-01 06:49:49.670,14536215,<p>You could iterate over the <code>mails</cod...,2020-10-28 13:57:53.453,10.256758,2021-09-01,2020-10,2021-09
1133,69009113,2021-09-01 06:59:57.993,14574199,"<p>Follow this <a href=""https://learn.microsof...",2020-11-04 01:32:02.717,10.040924,2021-09-01,2020-11,2021-09
1318,69009528,2021-09-01 07:32:39.687,16343464,<p>You can first create a table with the combi...,2021-06-29 14:57:08.477,2.123044,2021-09-01,2021-06,2021-09
...,...,...,...,...,...,...,...,...,...
3102578,77019266,2023-08-31 21:07:29.790,15211203,<p>Based on the current values of your <code>d...,2021-02-15 03:58:18.853,30.923824,2023-08-31,2021-02,2023-08
3102596,77019314,2023-08-31 21:15:16.417,14853083,<p>If you really want to set <strong>all</stro...,2020-12-18 23:33:07.173,32.863476,2023-08-31,2020-12,2023-08
3102679,77019489,2023-08-31 22:00:18.263,15261315,<p>Recursion 101: we need a base case. If <cod...,2021-02-22 17:00:43.327,30.673601,2023-08-31,2021-02,2023-08
3102725,77019611,2023-08-31 22:34:27.613,14868997,<p><em>1. Is Dapper rewriting the query when u...,2020-12-22 02:20:12.663,32.761441,2023-08-31,2020-12,2023-08


In [9]:
len(users_in_every_month)

26

### LSM

In [13]:
# answers (Old)
df_analysis = df_filtered.groupby(['owner_user_id', 'year_month'])['body'].agg(lambda x: ', '.join(map(str,x))).reset_index()
df_analysis

,owner_user_id,year_month,body
0,10008173,2021-09,<p>Since you already know the location on the ...
1,10008173,2021-10,<p>That <code>ENTRYPOINT</code> line doesn't f...
2,10008173,2021-11,<p>A Docker container only runs one process. ...
3,10008173,2021-12,<p>Connections between containers (over the Do...
4,10008173,2022-01,<p>You can use the Docker image name (<code>do...
...,...,...,...
18451,997358,2023-04,<p>To avoid various inefficiencies in your pro...
18452,997358,2023-05,<p>If you want an expression which is guarante...
18453,997358,2023-06,<p>How to optimize? Since you don't seem to h...
18454,997358,2023-07,"<p>Since the <a href=""/questions/tagged/jq"" cl..."


In [14]:
df_analysis.owner_user_id.nunique()

769

In [15]:
# Save individual body data into separate md file.
year_month = list(df_analysis['year_month'].unique().astype('str')) # get unique year_month
for i in range(len(year_month)):
    output_directory = f'/data1/StackOverflow/_Robustness/H4/lsm_data/oldbie/{year_month[i]}'
    os.makedirs(output_directory, exist_ok=True)
    data = df_analysis[df_analysis['year_month'] == year_month[i]]
    for index, row in data.iterrows():
        user_filename = f"{output_directory}/user_{row['owner_user_id']}.md"
        with open(user_filename, 'w') as md_file:
            md_file.write(f"## User {row['owner_user_id']}\n")
            md_file.write(row['body'] + '\n\n')
    print(f"Data has been saved to individual md files in the '{output_directory}' directory.")

Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/oldbie/2021-09' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/oldbie/2021-10' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/oldbie/2021-11' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/oldbie/2021-12' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/oldbie/2022-01' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/oldbie/2022-02' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/oldbie/2022-03' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/oldbie/2022-04' directory.
Data has been sa

In [16]:
# answers (new)
df_analysis2 = df_filtered2.groupby(['owner_user_id', 'year_month'])['body'].agg(lambda x: ', '.join(map(str,x))).reset_index()
df_analysis2.owner_user_id.nunique()

26

In [17]:
# Save individual body data into separate md file.
year_month = list(df_analysis2['year_month'].unique().astype('str')) # get unique year_month
for i in range(len(year_month)):
    output_directory = f'/data1/StackOverflow/_Robustness/H4/lsm_data/newbie/{year_month[i]}'
    os.makedirs(output_directory, exist_ok=True)
    data = df_analysis2[df_analysis2['year_month'] == year_month[i]]
    for index, row in data.iterrows():
        user_filename = f"{output_directory}/user_{row['owner_user_id']}.md"
        with open(user_filename, 'w') as md_file:
            md_file.write(f"## User {row['owner_user_id']}\n")
            md_file.write(row['body'] + '\n\n')
    print(f"Data has been saved to individual md files in the '{output_directory}' directory.")

Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/newbie/2021-09' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/newbie/2021-10' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/newbie/2021-11' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/newbie/2021-12' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/newbie/2022-01' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/newbie/2022-02' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/newbie/2022-03' directory.
Data has been saved to individual md files in the '/data1/StackOverflow/_Robustness/H4/lsm_data/newbie/2022-04' directory.
Data has been sa

### HERE, use separate py file for computing LSM
- compute.py for ob
- compute2.py for nb

#### 1) Newbie

In [35]:
# Newbie: LSM Score Aggregation
import os
import pandas as pd
folder_path = '/data1/StackOverflow/_Robustness/H4/lsm_result/newbie'
# Get a list of all files in the folder
file_list = os.listdir(folder_path)
file_list = sorted(file_list)
# Filter only pickle files
pickle_files = [file for file in file_list if file.endswith('.pickle')]

# Iterate through each pickle file and load it into a separate object
loaded_objects = pd.DataFrame(columns = ['User1', 'year_month', 'Similarity_toAvg'])
for pickle_file in pickle_files:
    file_path = os.path.join(folder_path, pickle_file)
    file_name = os.path.splitext(pickle_file)[0]
    # AGGREGATION
    with open(file_path, 'rb') as f:
        loaded_object = pickle.load(f)
    loaded_object['Similarity_toAvg'] = loaded_object['Similarity_toAvg'].astype(float)
    loaded_object = loaded_object.groupby('User1')['Similarity_toAvg'].mean().reset_index()
    loaded_object['year_month'] = file_name # Add year_month
    loaded_object = loaded_object[['User1', 'year_month', 'Similarity_toAvg']]
    # APPEND
    loaded_objects = pd.concat([loaded_objects, loaded_object])

/tmp/ipykernel_1898953/1001867863.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  loaded_objects = pd.concat([loaded_objects, loaded_object])


In [36]:
loaded_objects = loaded_objects.rename(columns = {"User1": "owner_user_id"})
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('.md', ''))
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('user_', ''))

In [37]:
# Add T_d (Treated: 2022-09 ~ 2023-08 (12 months) Cntrol: 2021-09 ~ 2022-08 (12 months) )
import numpy as np
T_d = np.zeros(len(loaded_objects), dtype = int)
T_d[300:] = 1
loaded_objects['T_d'] = T_d

# Make month variable
loaded_objects['month'] = loaded_objects['year_month'].str[5:].astype(int).astype(str)
# Add P_t (if  9, 10, 11 month == P_t == 0, the others P_t == 1)
loaded_objects['P_t'] = loaded_objects['month'].apply(lambda x: 0 if x in ['9', '10', '11'] else 1)
loaded_objects = loaded_objects.rename(columns={'Similarity_toAvg': 'LSM'})
loaded_objects['ln_LSM'] = np.log(loaded_objects['LSM'])
loaded_objects

,owner_user_id,year_month,LSM,T_d,month,P_t,ln_LSM
0,14460824,2021-09,0.170323,0,9,0,-1.770057
1,14536215,2021-09,0.162818,0,9,0,-1.815122
2,14574199,2021-09,0.166423,0,9,0,-1.793221
3,14624409,2021-09,0.175530,0,9,0,-1.739943
4,14672114,2021-09,0.162072,0,9,0,-1.819713
...,...,...,...,...,...,...,...
20,16540390,2023-08,0.167860,1,8,1,-1.784624
21,16578424,2023-08,0.166195,1,8,1,-1.794596
22,16764520,2023-08,0.163644,1,8,1,-1.810063
23,16895625,2023-08,0.185786,1,8,1,-1.683160


In [38]:
loaded_objects.describe()

,LSM,T_d,P_t,ln_LSM
count,600.000000,600.000000,600.000000,600.000000
mean,0.180807,0.500000,0.750000,-1.719627
std,0.029096,0.500417,0.433374,0.126928
min,0.153502,0.000000,0.000000,-1.874044
25%,0.166283,0.000000,0.750000,-1.794064
50%,0.174089,0.500000,1.000000,-1.748188
75%,0.184043,1.000000,1.000000,-1.692587
max,0.504248,1.000000,1.000000,-0.684687


In [39]:
loaded_objects.to_csv('lsm_new.csv', index = False)

#### 2) Oldbie

In [2]:
# Oldbie: LSM Score Aggregation
import os
import pandas as pd
import pickle
folder_path = '/data1/StackOverflow/_Robustness/H4/lsm_result/oldbie'
# Get a list of all files in the folder
file_list = os.listdir(folder_path)
file_list = sorted(file_list)
# Filter only pickle files
pickle_files = [file for file in file_list if file.endswith('.pickle')]

# Iterate through each pickle file and load it into a separate object
loaded_objects = pd.DataFrame(columns = ['User1', 'year_month', 'Similarity_toAvg'])
for pickle_file in pickle_files:
    file_path = os.path.join(folder_path, pickle_file)
    file_name = os.path.splitext(pickle_file)[0]
    # AGGREGATION
    with open(file_path, 'rb') as f:
        loaded_object = pickle.load(f)
    loaded_object['Similarity_toAvg'] = loaded_object['Similarity_toAvg'].astype(float)
    loaded_object = loaded_object.groupby('User1')['Similarity_toAvg'].mean().reset_index()
    loaded_object['year_month'] = file_name # Add year_month
    loaded_object = loaded_object[['User1', 'year_month', 'Similarity_toAvg']]
    # APPEND
    loaded_objects = pd.concat([loaded_objects, loaded_object])

/tmp/ipykernel_1944426/779855084.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  loaded_objects = pd.concat([loaded_objects, loaded_object])


In [3]:
loaded_objects = loaded_objects.rename(columns = {"User1": "owner_user_id"})
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('.md', ''))
loaded_objects['owner_user_id'] = loaded_objects['owner_user_id'].apply(lambda x: x.replace('user_', ''))

In [10]:
loaded_objects.iloc[9216:,]

,owner_user_id,year_month,Similarity_toAvg
0,10008173,2022-09,0.184744
1,10112124,2022-09,0.182984
2,10138734,2022-09,0.173257
3,10140124,2022-09,0.173601
4,10157127,2022-09,0.181486
...,...,...,...
763,9952196,2023-08,0.170453
764,9957710,2023-08,0.167970
765,99692,2023-08,0.172771
766,9971759,2023-08,0.170609


In [5]:
18432/2

9216.0

In [11]:
# Add T_d (Treated: 2022-09 ~ 2023-08 (12 months) Cntrol: 2021-09 ~ 2022-08 (12 months) )
import numpy as np
T_d = np.zeros(len(loaded_objects), dtype = int)

T_d[9216:] = 1 ### CHANGE THIS

loaded_objects['T_d'] = T_d

# Make month variable
loaded_objects['month'] = loaded_objects['year_month'].str[5:].astype(int).astype(str)
# Add P_t (if  9, 10, 11 month == P_t == 0, the others P_t == 1)
loaded_objects['P_t'] = loaded_objects['month'].apply(lambda x: 0 if x in ['9', '10', '11'] else 1)
loaded_objects = loaded_objects.rename(columns={'Similarity_toAvg': 'LSM'})
loaded_objects['ln_LSM'] = np.log(loaded_objects['LSM'])
loaded_objects

,owner_user_id,year_month,LSM,T_d,month,P_t,ln_LSM
0,10008173,2021-09,0.180097,0,9,0,-1.714260
1,10112124,2021-09,0.189455,0,9,0,-1.663603
2,10138734,2021-09,0.176147,0,9,0,-1.736439
3,10140124,2021-09,0.170093,0,9,0,-1.771410
4,10157127,2021-09,0.180470,0,9,0,-1.712189
...,...,...,...,...,...,...,...
763,9952196,2023-08,0.170453,1,8,1,-1.769295
764,9957710,2023-08,0.167970,1,8,1,-1.783972
765,99692,2023-08,0.172771,1,8,1,-1.755789
766,9971759,2023-08,0.170609,1,8,1,-1.768381


In [12]:
loaded_objects.describe()

,LSM,T_d,P_t,ln_LSM
count,18432.000000,18432.000000,18432.000000,18432.000000
mean,0.185506,0.500000,0.750000,-1.693061
std,0.028459,0.500014,0.433024,0.120027
min,0.164661,0.000000,0.000000,-1.803868
25%,0.171668,0.000000,0.750000,-1.762193
50%,0.177965,0.500000,1.000000,-1.726169
75%,0.188881,1.000000,1.000000,-1.666640
max,0.524787,1.000000,1.000000,-0.644763


In [13]:
loaded_objects.to_csv('lsm_old.csv', index = False)